<a href="https://colab.research.google.com/github/Souzantonio081/Desafios-Curso-DNC/blob/main/Tarefa_2_1_Prot%C3%B3tipo_de_Modelo_Preditivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

In [ ]:
# --- 1. Carregar Dados ---
# Carrega o arquivo CSV
df = pd.read_csv('Leads.csv')

In [ ]:
# --- 2. Preparação (Pre-processing) ---

# Mapear a variável alvo 'converteu_em_cliente' para 0 e 1
# 'sim' se torna 1 (classe positiva) e 'não' se torna 0 (classe negativa)
df['converteu_em_cliente'] = df['converteu_em_cliente'].map({'sim': 1, 'não': 0})

# Definir features (X) e alvo (y)
# Vamos dropar 'id_visitante' pois é apenas um identificador
X = df.drop(['converteu_em_cliente', 'id_visitante'], axis=1)
y = df['converteu_em_cliente']

# Identificar colunas numéricas e categóricas
numeric_features = ['tempo_navegacao_site', 'paginas_visitadas']
categorical_features = ['cidade_lead', 'setor_empresa']

In [ ]:
# --- 3. Divisão dos Dados (Train/Test Split) ---
# Dividir os dados em 70% para treino e 30% para teste
# Usar stratify=y é crucial por causa do desbalanceamento (75/25)
# Isso garante que os conjuntos de treino e teste tenham a mesma proporção de 'sim' e 'não'
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,    # 30% para teste
    random_state=42,  # Para reprodutibilidade
    stratify=y
)

In [ ]:
# --- 4. Criação do Pipeline de Pre-processing e Modelo ---

# Pipeline para features numéricas: Padronizar a escala (StandardScaler)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Pipeline para features categóricas: One-Hot Encoding
# handle_unknown='ignore' evita erros se uma nova categoria (ex: uma nova cidade)
# aparecer nos dados de teste ou em produção.
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar os pipelines de pré-processamento usando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
# --- 5. Criação do Pipeline Final com o Modelo ---
# O pipeline final irá: 1) pré-processar os dados, 2) treinar o classificador
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        # class_weight='balanced' é a decisão-chave aqui.
        # Ele ajusta o modelo para dar mais importância à classe
        # minoritária ('sim'), combatendo o desbalanceamento.
        class_weight='balanced',
        random_state=42,
        solver='liblinear'
    ))
])

In [ ]:
# --- 6. Treinamento do Modelo ---
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['tempo_navegacao_site',
                                                   'paginas_visitadas']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['cidade_lead',
                                                   'setor_empresa'])])),
                ('classifier',
                 LogisticRegression(class_weight='balanced', random_state=42,
                                    solver='liblinear'))])

In [ ]:
# --- 7. Avaliação do Modelo ---
# Fazer previsões nos dados de teste
y_pred = model.predict(X_test)
# Obter as probabilidades (scores) para a classe '1' (sim)
y_prob = model.predict_proba(X_test)[:, 1]

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred) # Foco na classe positiva (1)
recall = recall_score(y_test, y_pred)     # Foco na classe positiva (1)

In [ ]:
# --- 8. Apresentação das Métricas ---
print(f"Acurácia (Accuracy): {accuracy:.4f}")
print(f"Precisão (Precision): {precision:.4f}")
print(f"Revocação (Recall): {recall:.4f}")
print("\n--- Relatório de Classificação Detalhado ---")
print(classification_report(y_test, y_pred, target_names=['Não Converteu (0)', 'Converteu (1)']))

Acurácia (Accuracy): 0.9000
Precisão (Precision): 0.7500
Revocação (Recall): 0.8919

--- Relatório de Classificação Detalhado ---
                   precision    recall  f1-score   support

Não Converteu (0)       0.96      0.90      0.93       226
    Converteu (1)       0.75      0.89      0.81        74

         accuracy                           0.90       300
        macro avg       0.86      0.90      0.87       300
     weighted avg       0.91      0.90      0.90       300

